In [1]:
import torch
import torch.nn as nn
from torch.nn import Conv2d
import skimage
import skimage.io
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.transform import rescale
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.optim as optim
from torchvision import models
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import resnet18
import os
import skimage
import skimage.io
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from skimage.transform import rescale
from skimage import data
import pandas as pd
import PIL
import tensorflow as tf
from PIL import Image


from tqdm.notebook import trange, tqdm

from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
val_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomAffine(180),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
batch_size = 200

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [6]:
def eval_metrics(model, loader, criterion):
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output,label)


            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(loader)
            epoch_val_loss += val_loss/ len(loader)

In [7]:
model = torch.load("/content/drive/MyDrive/model.pt")

In [8]:
TEST_DIRECTORY = '/content/drive/MyDrive/data/test'

class Molecules_test(torch.utils.data.Dataset):
    def __init__(self, imgs, transforms = None):

        super().__init__()
        self.imgs = imgs
        self.transforms = transforms

    def __getitem__(self, idx):

        image_name = self.imgs[idx]

        img = Image.open(TEST_DIRECTORY + "/" + image_name)

        label_name = image_name.split('.')[0]

        ### Apply Transforms on image
        img = self.transforms(img)

        return img, label_name


    def __len__(self):
        return len(self.imgs)

In [9]:
tesimg = os.listdir(TEST_DIRECTORY)
#tesimg.remove('.ipynb_checkpoints')
tesset = Molecules_test(tesimg, val_transform)
tesloader = torch.utils.data.DataLoader(tesset, batch_size=batch_size,
                                         shuffle=False)

In [10]:
ans = pd.DataFrame(columns = ['img_id', 'class_id'])
with torch.no_grad():
    for data in tesloader:
        images, labels = data
        images = images.to(device)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        d = pd.DataFrame(columns = ['img_id', 'class_id'])
        d['img_id'] = labels
        d["class_id"] = [(int)(a) for a in predicted]
        ans = ans.append(pd.DataFrame(d,columns = ['img_id', 'class_id']))
print(ans)

     img_id class_id
0   hitest2        1
1   hitest1        1
2   hitest3        1
3   oxtest2        2
4   oxtest1        2
5   watest1        3
6   watest2        3
7   watest3        3
8   wrtest1        0
9   wrtest2        0
10  oxtest3        2
11  wrtest3        0


<ipython-input-10-08610a5035bb>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ans = ans.append(pd.DataFrame(d,columns = ['img_id', 'class_id']))
